In [1]:
#Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from citipy import citipy
import requests
from datetime import datetime
from config import weather_api_key
from scipy.stats import linregress
import matplotlib.pyplot as plt

url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [2]:
# Create a set of 1,500 random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs 

coordinates = list(lat_lngs)

# Create a list for holding the cities.

# Get the nearest city using the citipy module.
cities = []
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

629

In [3]:
# Perform an API call with the OpenWeatherMap.

# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")


city_data = []
record_count = 1
set_count = 1
    
# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city

    # Log the URL, record, and set numbers and the city.
    #print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1

# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_descr = city_weather["weather"][0]["main"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        if city_weather["weather"][0]["main"] == "Rain":
            city_rain = city_weather["rain"]["3h"]
        else:
            city_rain = 0
       
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        
        if city_weather["weather"][0]["main"] == "Snow":
            city_snow = city_weather["snow"]["3h"]
        else:
            city_snow = 0

        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Current Description": city_descr,
                          "Rain inches": city_rain,
                          "Snow inches": city_snow,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print(f"City not found. Skipping... {city} ")
        pass
            
# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")
print(len(city_data))

Beginning Data Retrieval     
-----------------------------
City not found. Skipping... clyde river 
City not found. Skipping... sitka 
City not found. Skipping... lolua 
City not found. Skipping... bambous virieux 
City not found. Skipping... belushya guba 
City not found. Skipping... norman wells 
City not found. Skipping... bargal 
City not found. Skipping... nguiu 
City not found. Skipping... oswego 
City not found. Skipping... mahebourg 
City not found. Skipping... rognan 
City not found. Skipping... iqaluit 
City not found. Skipping... halifax 
City not found. Skipping... jamestown 
City not found. Skipping... vaini 
City not found. Skipping... raudeberg 
City not found. Skipping... taolanaro 
City not found. Skipping... zabinka 
City not found. Skipping... yellowknife 
City not found. Skipping... nizhneyansk 
City not found. Skipping... tumannyy 
City not found. Skipping... mys shmidta 
City not found. Skipping... hihifo 
City not found. Skipping... umzimvubu 
City not found. Sk

In [4]:
city_data_df = pd.DataFrame(city_data)
new_order = ["City","Country","Date","Lat","Lng","Max Temp","Humidity","Cloudiness","Wind Speed","Current Description","Rain inches","Snow inches"]
city_data_df = city_data_df[new_order]
city_data_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches,Snow inches
0,Gombong,ID,2020-02-28 04:14:32,-7.61,109.51,91.85,59,54,1.66,Clouds,0.0,0.0
1,Ushuaia,AR,2020-02-28 04:14:32,-54.80,-68.30,46.40,81,40,36.91,Clouds,0.0,0.0
2,Bethel,US,2020-02-28 04:14:33,41.37,-73.41,33.80,42,20,17.22,Clouds,0.0,0.0
3,Salinopolis,BR,2020-02-28 04:14:33,-0.61,-47.36,80.73,76,93,14.81,Clouds,0.0,0.0
4,Klaksvik,FO,2020-02-28 04:14:33,62.23,-6.59,33.80,64,100,9.17,Clouds,0.0,0.0
5,Barrow,US,2020-02-28 04:14:34,71.29,-156.79,-36.40,90,1,10.29,Clear,0.0,0.0
6,Slave Lake,CA,2020-02-28 04:14:34,55.28,-114.77,37.40,64,75,5.82,Clouds,0.0,0.0
7,Ouargaye,BF,2020-02-28 04:14:35,11.50,0.06,75.34,14,2,10.11,Clear,0.0,0.0
8,Nome,US,2020-02-28 04:14:35,64.50,-165.41,1.40,77,1,0.98,Clear,0.0,0.0
9,Alyangula,AU,2020-02-28 04:14:35,-13.85,136.42,89.60,66,75,4.70,Clouds,0.0,0.0


In [ ]:
# Export the output file (CSV)
output_data_file = "Challenge/WeatherPy_challenge.csv"
city_data_df.to_csv(output_data_file, index_label="City_ID")